In [12]:
import numpy as np
import pandas as pd

from scipy import stats, linalg
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy 

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import RidgeCV    

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [13]:
df = pd.read_csv("./data/kc_house_data_FE.csv")

In [14]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,sqft_above,yr_built,...,grade#10,grade#11,grade#12,grade#13,zipcode#98004,zipcode#98102,zipcode#98109,zipcode#98112,zipcode#98039,zipcode#98040
0,221900.0,3,1.00,1180,5650,1.0,0,3,1180,1955,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,538000.0,3,2.25,2570,7242,2.0,0,3,2170,1951,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,180000.0,2,1.00,770,10000,1.0,0,3,770,1933,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,604000.0,4,3.00,1960,5000,1.0,0,5,1050,1965,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,510000.0,3,2.00,1680,8080,1.0,0,3,1680,1987,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21612 entries, 0 to 21611
Data columns (total 41 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   price                   21612 non-null  float64
 1   bedrooms                21612 non-null  int64  
 2   bathrooms               21612 non-null  float64
 3   sqft_living             21612 non-null  int64  
 4   sqft_lot                21612 non-null  int64  
 5   floors                  21612 non-null  float64
 6   waterfront              21612 non-null  int64  
 7   condition               21612 non-null  int64  
 8   sqft_above              21612 non-null  int64  
 9   yr_built                21612 non-null  int64  
 10  sqft_living15           21612 non-null  int64  
 11  sqft_lot15              21612 non-null  int64  
 12  presence_of_basement    21612 non-null  int64  
 13  presence_of_renovation  21612 non-null  int64  
 14  year                    21612 non-null

In [16]:
df.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,sqft_above,yr_built,...,grade#10,grade#11,grade#12,grade#13,zipcode#98004,zipcode#98102,zipcode#98109,zipcode#98112,zipcode#98039,zipcode#98040
count,2.161200e+04,21612.000000,21612.000000,21612.000000,2.161200e+04,21612.000000,21612.000000,21612.000000,21612.000000,21612.000000,...,21612.000000,21612.000000,21612.000000,21612.000000,21611.000000,21611.000000,21611.000000,21611.000000,21611.000000,21611.000000
mean,5.400835e+05,3.369471,2.114774,2079.921016,1.510739e+04,1.494332,0.007542,3.409356,1788.425319,1971.006247,...,0.052471,0.018462,0.004164,0.000602,0.014668,0.004859,0.005044,0.012447,0.002314,0.013049
std,3.671351e+05,0.907982,0.770177,918.456818,4.142142e+04,0.539991,0.086519,0.650668,828.094487,29.373636,...,0.222980,0.134618,0.064399,0.024519,0.120225,0.069536,0.070842,0.110874,0.048046,0.113487
min,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,1.000000,290.000000,1900.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.218375e+05,3.000000,1.750000,1426.500000,5.040000e+03,1.000000,0.000000,3.000000,1190.000000,1951.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.500000e+05,3.000000,2.250000,1910.000000,7.619000e+03,1.500000,0.000000,3.000000,1560.000000,1975.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6.450000e+05,4.000000,2.500000,2550.000000,1.068825e+04,2.000000,0.000000,4.000000,2210.000000,1997.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.700000e+06,11.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,5.000000,9410.000000,2015.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
X = df[[ col for col in df.columns if col != "price"]]
y = np.log(df["price"])

In [25]:
inds = pd.isnull(df).any(1).nonzero()[0]

AttributeError: 'Series' object has no attribute 'nonzero'

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=0)

In [22]:
linear_model = LinearRegression()
linear_model.fit(X_train,y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
y_pred=linear_model.predict(X_train)